# OPAN5505 Lab 3 Assignment

## Background

You've been engaged by the Federal Aviation Administration to perform an analysis on "strike" data. A "strike" refers to when an aircraft and an object within the air (typically wildlife) collide.

## Prepare your environment

You will need the `polars` package for this assignment. We'll also use `datetime` for date handling.

In [ ]:
# Import required libraries
import polars as pl
from datetime import datetime, date


## Load your dataset

Read the `faa_strikes.txt` dataset into Polars. Name the resulting DataFrame `strikes`.

In [ ]:
# Read the faa_strikes.txt data

strikes = pl.read_csv('faa_strikes.txt', separator='	', null_values=['NA'])
strikes


## Question 1: Running total of strikes by day through 2013

You are interested in seeing the running total of strikes over time (`Collision Date and Time`) on a daily basis. Using the `strikes` dataframe, create a new column that removes the time information but keeps the date information from the `Collision Date and Time` column (call this new field `date`). Next, aggregate the `Number of Strikes` by day (call this new field `daily_strikes`). Sort the data in ascending order by `date`. Then, create the running total of `daily_strikes` (name the new field `strikes_cumulative`) and filter to only the records **up to and including** the `date` of 2013-12-31. Name the resulting data frame `running_total_strikes`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [ ]:

# create a date column from the collision timestamp
running_total_strikes = (
    strikes
    .with_columns(
        # Parse the collision date time string into a  polars datetime and then extract the ddate
        pl.col("Collision Date and Time")
        .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%SZ", strict=False)
        .dt.date()
        .alias("date")
    )
    # the number of strikes by date
    .group_by("date")
    .agg(
        pl.col("Number of Strikes").sum().alias("daily_strikes")
    )
    # the result by date in ascending order
    .sort("date")
    # the running  total of daily strikes
    .with_columns(
        pl.col("daily_strikes").cum_sum().alias("strikes_cumulative")
    )
    # only dates up to and including 2013-12-31
    .filter(
        pl.col("date") <= date(2013, 12, 31)
    )
)
# resulting DataFrame
running_total_strikes


## Question 2: States with the third highest financial cost

The FAA is interested in the financial cost of strikes for each `Origin State`. Using the `strikes` dataframe: first, sum `Cost: Total $` by `Origin State` (name the resulting column `damage`) then `rank` these states in order of which state had the most financial `damage` (name the new column with rank information, `ranking`). Next, filter to show the row with the third highest `ranking`. Use a window function to answer this question. Name the resulting dataframe `damage_state`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [ ]:

# damage sum by Origin State and rank them from high to low
damage_state = (
    strikes
    .group_by("Origin State")
    .agg(
        # Sum the total cost; nulls are ignored
        pl.col("Cost: Total $").sum().alias("damage")
    )
    # Rank by damage in descending order
    .with_columns(
        pl.col("damage").rank(method="min", descending=True).alias("ranking")
    )
    # Filter for the third highest total damage
    .filter(pl.col("ranking") == 3)
)
damage_state


## Question 3: What are the second costliest Species Groups for each Aircraft Type?

The FAA wants to know if some species groups are more dangerous to particular types of aircraft. Using the `strikes` dataframe: first, sum financial damage (`Cost: Total $`) information by `Aircraft: Type` and `Wildlife: Species Group` (name the new field `damage`). Rank the rows within each `Aircraft: Type` based on the greatest amount of `damage`; when performing the ranking function, name the new column `ranking`. Return the rows that represent the `Wildlife: Species Group` values that caused the **second most** financial `damage` to each `Aircraft: Type`. Name the resulting dataframe `type_species`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a ranking function to receive credit on this question.*

In [ ]:

# cost sum by aircraft type and species group, then findd the second costliest group per type
type_species = (
    strikes
    .group_by(["Aircraft: Type", "Wildlife: Species Group"])
    .agg(
        pl.col("Cost: Total $").sum().alias("damage")
    )
    # each aircraft type by damge in descending order
    .with_columns(
        pl.col("damage")
        .rank(method="min", descending=True)
        .over("Aircraft: Type")
        .alias("ranking")
    )
    # select the rows corresponding to the second highest damage per Aircraft Type.
    .filter(pl.col("ranking") == 2)
)
type_species


## Question 4: Which days had the greatest positive jump in strikes?

The FAA wants to investigate which days had the largest increase in strikes from the previous day. Using the `strikes` dataframe: sum the `Number of Strikes` measure by day (note: not by day and time). You can use a similar technique from Question 1 to create a new column from `Collision Date and Time` without time information and with date information (name this column `date`). Next, compute the previous day's strikes using a window function (name the new column `previous_day`), and calculate the difference of strikes between the current day and the previous day (name the new column `delta_strikes`). Sort the resulting data by `delta_strikes` in descending order to find which days had the highest increase of strikes from the previous day. Name the resulting dataframe `greatest_strike_increase`.

**Hint:** This code is more involved that other exercises, so I will explicitly name the steps to answer the question.

1. Start with the `strikes` data frame
2. Create a new column called `date` which removes the time information in the `Collision Date and Time` column
3. Sum the `Number of Strikes` by `date`, and name the new column `daily_strikes`
4. Sort by the `date` column in ascending order
5. Create a new column called `previous_day` to calculate what the `daily_strikes` were in the previous day
6. Create a new column called `delta_strikes` which subtracts `previous_day` from `daily_strikes`
7. Sort in descending order by `delta_strikes`

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [ ]:

# the days with the greatest positive jump in strikes across all aircraft
greatest_strike_increase = (
    strikes
    # a date column without time
    .with_columns(
        pl.col("Collision Date and Time")
        .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%SZ", strict=False)
        .dt.date()
        .alias("date")
    )
    # the number of strikes by date
    .group_by("date")
    .agg(
        pl.col("Number of Strikes").sum().alias("daily_strikes")
    )
    # Sort by date
    .sort("date")
    # the previous days strikes using a window function 
    .with_columns(
        pl.col("daily_strikes").shift(1).alias("previous_day")
    )
    # the difference between the current day and the previous day
    .with_columns(
        (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
    )
    # descending to find the greatest positive jump
    .sort("delta_strikes", descending=True)
)
greatest_strike_increase


## Question 5: Which single day had the greatest increase in strikes for each `Aircraft: Type`?

FAA was interested in the exercise from the last question but now wants to determine the largest delta between days for each `Aircraft: Type`. What are the days that had largest positive change in strikes for each `Aircraft: Type`? In order to answer this question, you will need to perform the exercise from Question 4, but this time include a grouping by `Aircraft: Type`. After grouping by `Aircraft: Type`, add a column called `ranking` and use a window function to determine the day with the largest increase in strikes from the previous day. Name the resulting data frame `greatest_strike_increase_type`

NOTE: Helicopters are not struck with a high frequency and will not show up in your analysis; this is fine.

In [ ]:

# for each aircraft typ the day with the greatest increase in stikeds from the previous day
greatest_strike_increase_type = (
    strikes
    # making date column without time
    .with_columns(
        pl.col("Collision Date and Time")
        .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%SZ", strict=False)
        .dt.date()
        .alias("date")
    )
    # strikes by Aircraft Type and date.
    .group_by(["Aircraft: Type", "date"])
    .agg(
        pl.col("Number of Strikes").sum().alias("daily_strikes")
    )
    # within each aircraft type by date
    .sort(["Aircraft: Type", "date"])
    # previous day's strikes per aircraft type
    .with_columns(
        pl.col("daily_strikes")
        .shift(1)
        .over("Aircraft: Type")
        .alias("previous_day")
    )
    # the daily difference
    .with_columns(
        (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
    )
    # delta within each aircraft type to identify the greatest increase.
    .with_columns(
        pl.col("delta_strikes")
        .rank(method="min", descending=True)
        .over("Aircraft: Type")
        .alias("ranking")
    )
    # retain only the days with the greatest increase for each aircraft type
    .filter(pl.col("ranking") == 1)
)
greatest_strike_increase_type
